In [1]:
pip install tk 

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import Label, ttk, messagebox
from tkinter import font
from tkinter.constants import DISABLED, NORMAL
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
#from requests.api import post
#from bs4.element import Tag

# create a window window
window = tk.Tk()
 
# set the title of window window
window.title("Web Scrapper")

 
# set the configuration of window window
window.geometry("400x200")
#window.resizable(0, 0)

# get the website from the user
def get_website():
    def linkedin():
        button1.destroy()
        button2.destroy()
        

        def back_to_website():
            click_label.destroy()
            scrap_button.destroy()
            download_button.destroy()
            back_button.destroy()
            get_website()

        def ExtractSectionUrl():
            the_url= "https://www.linkedin.com/content-hub/?trk=homepage-basic_guest_nav_menu_discover"

            data = requests.get(the_url).content #get the contents of the page using get method
            MyParser = BeautifulSoup(data,'lxml')

            SectionTags = MyParser.find("section",{"class":"explore-section"}).find_all('a')

            SectionLinks=[]#find Section URLS and store in list
            for links in SectionTags:
                SectionLinks.append(links.get('href'))#append every link in the list
                print("getting section URLS...")
            ExtractPostURL(SectionLinks) #function call in order to pass the Section Links to the ExtractPostURL funtion

        def ExtractPostURL(SectionLinks):
            global ProfileLinks
            SectionURL = SectionLinks
            ProfileLinks = [] #to store the links of the profiles
            for link in SectionURL:#get the contents of every urls
                pagedata = requests.get(link).content
                MySoup = BeautifulSoup(pagedata,'lxml')

                Posttag = MySoup.find_all("h3",{"class":"share-update-card__actor-text"})
                for n in range(len(Posttag)):
                    atag = Posttag[n].find("a")
                    print("extracting profiles... \nplease wait for some seconds")
                    ProfileLinks.append(atag.get("href"))
                    n=n+1

        def scrap():
            click_label.destroy()
            scrap_button.config(state=DISABLED)
            ExtractSectionUrl()
            time.sleep(15)
            download_button.config(state=NORMAL)
            file_ready_label = Label(window, text="File Ready!", font=('Segoe UI',17, font.BOLD))
            file_ready_label.place(x=110,y=40)

        def download():
            
            with open("profilelinks.csv",'w') as f:
                f.write(str(ProfileLinks))
                f.close

            messagebox.showinfo("info", "File downloaded successfully!")
            

        click_label = Label(window, text="Click on Scrap Data\n to start scraping", font=('Segoe UI',17, font.BOLD))
        click_label.place(x=90,y=40)

        scrap_button = ttk.Button(window, text="Scrap Data", command=scrap, width=10)
        scrap_button.place(x=115,y=140)
        scrap_button.focus_set()

        back_button = ttk.Button(window, text="Back", command=back_to_website, width=10)
        back_button.place(x=30,y=140)
        download_button = ttk.Button(window, text="Download", command=download, width=10, state=DISABLED)
        download_button.place(x=200,y=140)
        
    def naukri():
        button1.destroy()
        button2.destroy()
        
        def back_to_website():
            click_label.destroy()
            scrap_button.destroy()
            download_button.destroy()
            back_button.destroy()
            get_website()

        def extract_profile():
            global profile_data, urls
            url='https://www.naukri.com/python-developer-recruiters-in-bangalore'
            r=requests.get(url) #get - extracts the content from the website url

            urls=list() #list of all profile urls
            soup = BeautifulSoup(r.text,'html.parser') #parsing website main page

            for a in soup.find_all('a',{'class':'ellipsis','target':'_blank'}):
                urls.append(a['href']) #adds all url of people into a list
                print("getting URLS of people...")
            #----- main page parsing is over------
            #----- individual page parsing starts----
                
            profile_data = list()
            for url in urls:
                profile=dict()
                print("getting datas...")
                page_soup=BeautifulSoup(requests.get(url).text,'html.parser') #parsing individual profile page
                name=page_soup.find('h1',{'class':'fl ellipsis wLimit hd'})
                profile['name']=name.text #getting name of individual
                role=page_soup.find('div',{'class':'ellipsis'})
                profile['role']=role.text #role of individual
                Company_Name=page_soup.find('a',{'class':'fl ellipsis widLrg'})
                profile['company_name']=Company_Name.text #company of individual
                profile['location']='Bangalore'
                profile_data.append(profile) #adding name,role,company,location to list

        def scrap():
            click_label.destroy()
            scrap_button.config(state=DISABLED)
            scraping_label = Label(window, text="Scrapping data...", font=('Segoe UI',17, font.BOLD))
            scraping_label.place(x=70,y=20)
            extract_profile()
            time.sleep(15)
            download_button.config(state=NORMAL)
            scraping_label.destroy()
            
            file_naukri = Label(window, text="File Ready!", font=('Segoe UI',17, font.BOLD))
            file_naukri.place(x=110,y=40)

        def download():

            profiles = pd.DataFrame(profile_data) #converting list into a dataframe( similar to excel)
            urls_df=pd.DataFrame(urls,columns=['profile_url']) #column name for urls
            naukri_scrape=pd.concat([profiles,urls_df],axis=1)
            naukri_scrape.to_csv('naukri_scrape.csv')

            messagebox.showinfo("info", "File downloaded successfully!")
            

        click_label = Label(window, text="Click on Scrap Data\n to start scraping", font=('Segoe UI',17, font.BOLD))
        click_label.place(x=90,y=40)

        scrap_button = ttk.Button(window, text="Scrap Data", command=scrap, width=10)
        scrap_button.place(x=115,y=140)
        scrap_button.focus_set()

        back_button = ttk.Button(window, text="Back", command=back_to_website, width=10)
        back_button.place(x=30,y=140)
        download_button = ttk.Button(window, text="Download", command=download, width=10, state=DISABLED)
        download_button.place(x=200,y=140)

    button1 = tk.Button(window, text="LinkedIn", command=linkedin, width=10, font=('Segoe UI',20))
    button2 = tk.Button(window, text="Naukri.com", command=naukri, width=10, font=('Segoe UI',20))
    button1.place(x=80,y=20)
    button2.place(x=80,y=100)

    quit_button = ttk.Button(window, text="Quit", command=quit, width=10)
    quit_button.place(x=285, y=140)

get_website()


window.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Sohan\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-2-38827f888da8>", line 69, in scrap
    ExtractSectionUrl()
  File "<ipython-input-2-38827f888da8>", line 43, in ExtractSectionUrl
    SectionTags = MyParser.find("section",{"class":"explore-section"}).find_all('a')
AttributeError: 'NoneType' object has no attribute 'find_all'
